#Aufgabe 4

Wir verwenden das Sentiment 140 Dataset, welches mit tfds heruntergeladen werden kann

In [1]:
!pip install tfds-nightly
!pip install emot

     |████████████████████████████████| 3.9MB 37.3MB/s 


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import pickle 

#download dataset
train, test = tfds.load('Sentiment140', split=['train', 'test'], batch_size=-1, as_supervised=True)

Dataset sentiment140 downloaded and prepared to /root/tensorflow_datasets/sentiment140/1.0.0. Subsequent calls will reuse this data.


In [3]:
#convert to numpy array
X_train, y_train = tfds.as_numpy(train)
X_test, y_test = tfds.as_numpy(test)

## b + c) Vorverarbeitung

- Hyperlinks entfernen		https://teco.kit.edu/
- Emojis in Wörter konvertieren 	⚽ => ‘soccerball’
- Emoticons entfernen 		:) :D :( :P
- Mentions entfernen 		@tecoKIT
- Zahlen entfernen			1234
- Satzzeichen entfernen 		.,-:
- Whitespace zu einfachem Leerzeichen konvertieren
- Resultierenden Text in “Tokens” umwandeln (jedes Wort separat betrachten)
- Lemmatisierung			looking => look, went => go
- Stoppwörter entfernen		the, is, at, which

## c) What is lemmatization
Bei der Lemmatisierung werden Wörter in ihren Wortstamm umgewandelt.
Hierbei wird nicht einfach, wie beim "word stemming" die Wortendung abgeschnitten, sondern es wird der korrekte Wortstamm mithilfe eines Wörterbuches verwendet.

Lemmatisierung ist zwar besser als "word stemming", jedoch nicht perfekt. Zum Beispiel kann "meeting" als Substantiv "meeting" oder als present participle des Verbs "meet" verstanden werden.

Beispiele: 
- studies => study
- studying => study
- better => good
- met => meet
- meeting => meeting

In [ ]:
# get wordlist for stopwords
import nltk
nltk.download('stopwords') 
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
print(X_test[1])
print(y_test[1])

b'SOOO DISSAPOiNTED THEY SENT DANNY GOKEY HOME... YOU STiLL ROCK ...DANNY ... MY HOMETOWN HERO !! YEAH MiLROCKEE!!'
4


In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize.casual import TweetTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from emot.emo_unicode import UNICODE_EMO, EMOTICONS
import string

stop_words = stopwords.words('english')
tokenizer = TweetTokenizer()
lemmatiser = WordNetLemmatizer()
emoji_pattern = re.compile("["
                        u"\U0001F600-\U0001F64F" # emoticons
                        u"\U0001F300-\U0001F5FF" # symbols & pictographs
                        u"\U0001F680-\U0001F6FF" # transport & map symbols
                        u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251"
                        "]+", flags=re.UNICODE)
emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')

def remove_hyperlinks(text):
    return re.sub(r'http\S+', '', text)

def remove_emoticons(text):
    return emoticon_pattern.sub(r'', text)

def clean_text(text):
    # remove mentions
    text_no_mention = re.sub(r'@\w*', '', text)
    # remove numbers
    text_nonum = re.sub(r'\d+', '', text_no_mention)
    # remove punctuations and convert characters to lower case
    text_nopunct = "".join([char.lower() for char in text_nonum if char not in string.punctuation]) 
    # substitute multiple whitespace with single whitespace
    # Also, removes leading and trailing whitespaces
    text_no_doublespace = re.sub('\s+', ' ', text_nopunct).strip()

    return text_no_doublespace

def remove_emoji(text):
    return emoji_pattern.sub(r'', text)

# Converting emojis to words
def convert_emojis(text):
    for emot in UNICODE_EMO:
        text = text.replace(emot, UNICODE_EMO[emot].replace(":"," "))
    return text

def convert_emoticons(text):
    for emot in EMOTICONS:
        text = re.sub(u'('+emot+')', "_".join(EMOTICONS[emot].replace(",","").split()), text)
        return text

def preprocess_text(text, emoji_convert=True):
    text = remove_hyperlinks(text)
    text = convert_emojis(text)
    text = remove_emoticons(text)
    text = clean_text(text)
    # Tokenise words with tweet tokenizer - better than Regexptokenizer
    tokens = tokenizer.tokenize(text)
    
    # Lowercase and lemmatise 
    lemmas = [lemmatiser.lemmatize(token.lower(), pos='v') for token in tokens]
    
    # Remove stopwords
    keywords= [lemma for lemma in lemmas if lemma not in stop_words]
    print(keywords)
    return keywords

In [ ]:
test = "Hello @tecoKIT :) :D I loved! !! the football match :D ⚽⚽⚽ https://www.youtube.com/watch?v=dQw4w9WgXcQ"
print(preprocess_text(test))

['hello', 'love', 'football', 'match', 'soccerball', 'soccerball', 'soccerball']
['hello', 'love', 'football', 'match', 'soccerball', 'soccerball', 'soccerball']


In [ ]:
X_train_tokens = [preprocess_text(x.decode("utf-8")) for x in X_train]

In [ ]:
# Save to disk
pickle.dump(X_train_tokens, open("X_train_tokens.pkl","wb"))

## d) What are word embeddings?

Als Worteinbettung wird die Repräsentation von Wörtern in einem Vektorraum bezeichnet.
Jedem Wort wird ein Vektor zugeordnet.


## e) Transformer Model (Tensorflow)

In [ ]:
!pip install transformers

In [ ]:
# optionaly load
#X_train_tokens = pickle.load(open("X_train_tokens.pkl","rb"))

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer2 = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
X_train2 = pd.DataFrame([' '.join(x) for x in X_train_tokens]).dropna()
X_train2 = X_train2[0].apply(lambda x: tokenizer2.encode(x))

In [ ]:
max_len = 0
for i in X_train_list_again.values:
    if len(i) > max_len:
        max_len = len(i)

X_train_padded = np.array([i + [0]*(max_len-len(i)) for i in X_train2.values])

In [ ]:
pickle.dump(X_train_padded, open("X_train_padded.pkl","wb"))

In [ ]:
import pickle
import pandas as pd
import numpy as np
# optionally load
# padded = pickle.load(open("X_train_padded.pkl","rb"))

In [3]:
attention_mask = np.where(X_train_padded != 0, 1, 0)
attention_mask.shape

(1600000, 290)

In [ ]:
import torch
from transformers import DistilBertModel
import tensorflow as tf
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

input_ids = torch.tensor(padded)  


In [ ]:
attention_mask = torch.tensor(attention_mask)

In [ ]:
with torch.no_grad():
  last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
X_train_features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
from sklearn.linear_model import LogisticRegression
reg = LogisticRegression()
reg.fit(X_train_features, y_train)

Leider stürzt Google Colab wegen zu hoher RAM Auslastung ab, wenn folgende Zeile ausgeführt wird

    last_hidden_states = model(input_ids, attention_mask=attention_mask)



## e.2) Tf-IDF

Da wir aufgrund der Beschränkungen von Google Colab mit Transformern nicht weitergekommen sind, haben wir uns mit Tf-IDF (term-frequency times inverse document-frequency) beschäftigt

Alternative zu Worteinbettungen (Word2Vec)

Datensatz wird als spärliche Matrix dargestellt. Die Zeilen sind die Datenpunkte (Tweets), die Spalten sind die alle Wörter, die in dem Datensatz vorkommen.

Wenn ein Datenpunkt ein Wort nicht enthält, so ist der Wert in der Matrix 0. Wenn ein Datenpunkt ein Wort enthält, so ist, abhängig von der Häufigkeit des Wortes im Datenpunkt und im Datensatz ein Wert zwischen 1 und 0 vorhanden.

In [ ]:
# Import packages and modules
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

# Create an instance of TfidfVectorizer
vectoriser = TfidfVectorizer(analyzer=preprocess_text)

In [ ]:
# Fit to the data and transform to feature matrix
X_train = vectoriser.fit_transform(X_train)

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
['clt', 'airport', 'get', 'ready', 'fly', 'mo', 'grandfathers', 'funeral', 'look', 'forward', 'see', 'family', 'miss', 'craig', 'amp', 'kid', 'already']
['goodnight', 'twitter', 'btw', 'god', 'give', 'good', 'classmates', 'lol']
['soooooooooo', 'cant', 'find', 'glass']
['awesome', 'day', 'good', 'time', 'hang', 'danimal', 'great', 'ride', 'coast', 'fun', 'ahem', 'even', 'event']
['editor', 'name', 'vince', 'make', 'corrections', 'didnt', 'make', 'wince', 'cool', 'book', 'really', 'happen']
['remember', 'monthold', 'right', 'longer', 'control', 'sleep', 'schedule']
['quothow', 'beautiful', 'unfailing', 'love', 'never', 'change', 'god', 'remain', 'holy', 'one', 'unfailing', 'lovequot']
['welcome', 'professional', 'life']
['im', 'always', 'fine', 'never', 'allow', 'anybody', 'anything', 'get']
['work']
['tire', 'stuff', 'work', 'sammy']
['lmfaooooooo', 'pleaseeeeee', 'im', 'already', 'scar', 'heart', 'dat', 'u', 'im', 'fli

In [ ]:
# Convert sparse matrix to dataframe
X_train = pd.DataFrame.sparse.from_spmatrix(X_train)

In [ ]:
# Save mapping on which index refers to which words
col_map = {v:k for k, v in vectoriser.vocabulary_.items()}

In [ ]:
word_column = pd.DataFrame.from_dict(col_map, orient='index').sort_index()[0]

In [ ]:
word_column.head(50)

In [ ]:
X_train = X_train.rename(columns=word_column)

(1600000, 412149)

In [ ]:
#save it
pickle.dump(vectoriser.vocabulary_, open("vectorizer.pkl","wb"))
pickle.dump(vectoriser, open("vectorizerdef.pkl","wb"))
pickle.dump(X_train, open("X_train.pkl","wb"))

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# optionally load
X_train = pickle.load(open("/content/drive/MyDrive/X_train_new.pkl","rb"))
X_train = pickle.load(open("X_train.pkl","rb"))

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(random_state=123)
sgd_scores = cross_val_score(sgd, X_train, y_train, cv=5)
print(sgd_scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (sgd_scores.mean(), sgd_scores.std() * 2))

Leider hat bei diesem Ansatz das Training zu lange gedauert, und es kam zu einem Timeout

## g) Advantages/Disadvantages

Wie bereits in Aufgabe 3h beschrieben, neigen RNNs durch die Rekursion beim Training zu vanishing/exploding Gradients.

Bei Transformern läuft das Training jedoch nicht nacheinander/rekursiv ab, sondern parallel. Bei der Betrachtung von natürlicher Sprache wird nicht Wort für Wort, sondern beispielsweise ein ganzer Satz auf einmal betrachtet.

## h) Attention - Aufmerksamkeit


Ashish Vaswani et al. haben in dem Paper "Attention Is All You Need" erstmals Transformer vorgestellt.

Aufmerksamkeit beim maschinellen Lernen wurde jedoch auch bereits vorher bei anderen Modellen verwendet, ursprünglich für die maschinelle Übersetzung von Text in eine andere Sprache.

Aufmerksamkeit bezeichnet beim maschinellen Lernen die Möglichkeit, wichtige von unwichtigen Daten zu unterscheiden um den Fokus auf die wichtigeren (Teil-)Daten zu legen.

Aufmerksamkeit wird mit 3 Parametern berechnet:
- $Q$ Anfrage
- $K$ Schlüssel
- $V$ Wert

Bildquellen: https://towardsdatascience.com/attention-and-its-different-forms-7fc3674d14dc


### Multi-Head attention

Bei Multi-Head-Attention werden die Aufmerksamkeitsmechanismen parallel ausgeführt.
$$ \text{MultiHead}\left(\textbf{Q}, \textbf{K}, \textbf{V}\right) = \left[\text{head}_{1},\dots,\text{head}_{h}\right]\textbf{W}_{0} $$
$$ \text{where} \text{ head}_{i} = \text{Attention} \left(\textbf{Q}\textbf{W}_{i}^{Q}, \textbf{K}\textbf{W}_{i}^{K}, \textbf{V}\textbf{W}_{i}^{V} \right) $$

### Scaled Dot-Product (von Transformern verwendet)

$$ {\text{Attention}}(Q, K, V) = \text{softmax}(\frac{QK^{T}}{\sqrt{d_k}})V $$